## Segmenting and Clustering Neighborhoods in Toronto

In [12]:
# Installing necessary packages
!pip install BeautifulSoup4
!pip install requests
!pip install geopy
!pip install folium

You should consider upgrading via the 'c:\users\jeevan\appdata\local\programs\python\python37-32\python.exe -m pip install --upgrade pip' command.


- Getting Data from wikipedia
- parsing html data using BeautifulSoup
- storing the data into Pandas DataFrame

In [4]:
#imports
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [5]:
#pulling data from wiki page and creating soup object
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(source.text, 'html.parser')

#iterating to get the data from htm page and store it into a list
data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
        
    if(index == 0):
        columns = section
    else:
        data.append(section)

#converting the available data into Pandas DataFrame
toronto_df = pd.DataFrame(data = data, columns = columns)
toronto_df.head()

Postal Code           Borough               Neighborhood
0         M1A      Not assigned               Not assigned
1         M2A      Not assigned               Not assigned
2         M3A        North York                  Parkwoods
3         M4A        North York           Victoria Village
4         M5A  Downtown Toronto  Regent Park, Harbourfront

- Removing the 'Not assigned' in Boroughts

In [6]:
#Remove Boroughts that are 'Not assigned'
toronto_df = toronto_df[toronto_df['Borough'] != 'Not assigned']
toronto_df.head()

Postal Code           Borough                                 Neighborhood
2         M3A        North York                                    Parkwoods
3         M4A        North York                             Victoria Village
4         M5A  Downtown Toronto                    Regent Park, Harbourfront
5         M6A        North York             Lawrence Manor, Lawrence Heights
6         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government

- More than one neighborhood can exist in one postal code area, combined these into one row with the neighborhoods separated with a comma
- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [7]:
toronto_df["Neighborhood"] = toronto_df.groupby("Postal Code")["Neighborhood"].transform(lambda neigh: ','.join(neigh))

toronto_df = toronto_df.drop_duplicates()

if(toronto_df.index.name != 'Postal Code'):
    toronto_df = toronto_df.set_index('Postal Code')

toronto_df.head()

Borough                                 Neighborhood
Postal Code                                                               
M3A                North York                                    Parkwoods
M4A                North York                             Victoria Village
M5A          Downtown Toronto                    Regent Park, Harbourfront
M6A                North York             Lawrence Manor, Lawrence Heights
M7A          Downtown Toronto  Queen's Park, Ontario Provincial Government

In [8]:
toronto_df['Neighborhood'].replace("Not assigned",toronto_df["Borough"], inplace=True)
toronto_df.head()

Borough                                 Neighborhood
Postal Code                                                               
M3A                North York                                    Parkwoods
M4A                North York                             Victoria Village
M5A          Downtown Toronto                    Regent Park, Harbourfront
M6A                North York             Lawrence Manor, Lawrence Heights
M7A          Downtown Toronto  Queen's Park, Ontario Provincial Government

- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe

In [9]:
toronto_df.shape

(103, 2)

### Adding Geospatial data

In [9]:
#loading data lat/lon data from the csv file
lat_lng_coords_df = pd.read_csv("Geospatial_Coordinates.csv")

#converting index to postal code
lat_lng_coords_df.columns = ["Postal Code", "Latitude", "Longitude"]
if(lat_lng_coords_df.index.name != 'Postal Code'):
    lat_lng_coords_df = lat_lng_coords_df.set_index('Postal Code')
    
lat_lng_coords_df.head()

Latitude  Longitude
Postal Code                      
M1B          43.806686 -79.194353
M1C          43.784535 -79.160497
M1E          43.763573 -79.188711
M1G          43.770992 -79.216917
M1H          43.773136 -79.239476

In [10]:
toronto_df = toronto_df.join(lat_lng_coords_df)
toronto_df.head(11)

Borough                                 Neighborhood  \
Postal Code                                                                  
M3A                North York                                    Parkwoods   
M4A                North York                             Victoria Village   
M5A          Downtown Toronto                    Regent Park, Harbourfront   
M6A                North York             Lawrence Manor, Lawrence Heights   
M7A          Downtown Toronto  Queen's Park, Ontario Provincial Government   
M9A                 Etobicoke      Islington Avenue, Humber Valley Village   
M1B               Scarborough                               Malvern, Rouge   
M3B                North York                                    Don Mills   
M4B                 East York              Parkview Hill, Woodbine Gardens   
M5B          Downtown Toronto                     Garden District, Ryerson   
M6B                North York                                    Glencairn   

              Latitude  Longitude  
Postal Code                        
M3A          43.753259 -79.329656  
M4A          43.725882 -79.315572  
M5A          43.654260 -79.360636  
M6A          43.718518 -79.464763  
M7A          43.662301 -79.389494  
M9A          43.667856 -79.532242  
M1B          43.806686 -79.194353  
M3B          43.745906 -79.352188  
M4B          43.706397 -79.309937  
M5B          43.657162 -79.378937  
M6B          43.709577 -79.445073

### Exploring and Clustering the Toronto neighborhoods

In [13]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [14]:
toronto_f_df = toronto_df[toronto_df['Borough'].str.contains('Toronto')]
toronto_f_df.head()

Borough                                 Neighborhood  \
Postal Code                                                                  
M5A          Downtown Toronto                    Regent Park, Harbourfront   
M7A          Downtown Toronto  Queen's Park, Ontario Provincial Government   
M5B          Downtown Toronto                     Garden District, Ryerson   
M5C          Downtown Toronto                               St. James Town   
M4E              East Toronto                                  The Beaches   

              Latitude  Longitude  
Postal Code                        
M5A          43.654260 -79.360636  
M7A          43.662301 -79.389494  
M5B          43.657162 -79.378937  
M5C          43.651494 -79.375418  
M4E          43.676357 -79.293031

- Generating map visualisation

In [16]:
#Show an initial map of the neighborhoods in Toronto
# create map of Toronto using first entries latitude and longitude values
map_toronto = folium.Map(location=[toronto_f_df["Latitude"][0], toronto_f_df["Longitude"][0]], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_f_df['Latitude'], toronto_f_df['Longitude'], toronto_f_df['Borough'], toronto_f_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [17]:
CLIENT_ID = '4VCMW1XE2ZLMEMOFG1WTCVN3BCM3K2YA1QG0HDJYRVKLYEW4' # your Foursquare ID
CLIENT_SECRET = 'L1KFTXJFXIN0C2S0C14J5RPEEPYOKZ5X1XSC4R3RY2X0O5UB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4VCMW1XE2ZLMEMOFG1WTCVN3BCM3K2YA1QG0HDJYRVKLYEW4
CLIENT_SECRET:L1KFTXJFXIN0C2S0C14J5RPEEPYOKZ5X1XSC4R3RY2X0O5UB


In [19]:
# Get data of first neighborhood
neighborhood_latitude = toronto_df['Latitude'][0] # neighborhood latitude value
neighborhood_longitude = toronto_df['Longitude'][0] # neighborhood longitude value

neighborhood_name = toronto_df['Neighborhood'][0] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


In [20]:
# Setup API URL to explore venues near by
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)
neighborhood_json = requests.get(url).json()

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues = neighborhood_json['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

name                  categories        lat  \
0                    Brookbanks Park                        Park  43.751976   
1                      Variety Store           Food & Drink Shop  43.751974   
2  Corrosion Service Company Limited  Construction & Landscaping  43.752432   

         lng  
0 -79.332140  
1 -79.333114  
2 -79.334661

In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Get Venue Data from Foursquare

In [23]:
toronto_venues_df = getNearbyVenues(names=toronto_df['Neighborhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [24]:
print(toronto_venues_df.shape)
toronto_venues_df.head()

(2130, 7)


Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0         Parkwoods              43.753259              -79.329656   
1         Parkwoods              43.753259              -79.329656   
2         Parkwoods              43.753259              -79.329656   
3  Victoria Village              43.725882              -79.315572   
4  Victoria Village              43.725882              -79.315572   

                               Venue  Venue Latitude  Venue Longitude  \
0                    Brookbanks Park       43.751976       -79.332140   
1                      Variety Store       43.751974       -79.333114   
2  Corrosion Service Company Limited       43.752432       -79.334661   
3             Victoria Village Arena       43.723481       -79.315635   
4                          Portugril       43.725819       -79.312785   

               Venue Category  
0                        Park  
1           Food & Drink Shop  
2  Construction & Landscaping  
3                Hockey Arena  
4       Portuguese Restaurant

In [25]:
toronto_venues_df.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
Agincourt                                                               5   
Alderwood, Long Branch                                                  8   
Bathurst Manor, Wilson Heights, Downsview North                        20   
Bayview Village                                                         4   
Bedford Park, Lawrence Manor East                                      23   
Berczy Park                                                            56   
Birch Cliff, Cliffside West                                             5   
Brockton, Parkdale Village, Exhibition Place                           24   
Business reply mail Processing Centre, South Ce...                     16   
CN Tower, King and Spadina, Railway Lands, Harb...                     16   
Caledonia-Fairbanks                                                     4   
Canada Post Gateway Processing Centre                                  13   
Cedarbrae                                                               8   
Central Bay Street                                                     64   
Christie                                                               17   
Church and Wellesley                                                   75   
Clarks Corners, Tam O'Shanter, Sullivan                                15   
Cliffside, Cliffcrest, Scarborough Village West                         2   
Commerce Court, Victoria Hotel                                        100   
Davisville                                                             33   
Davisville North                                                        9   
Del Ray, Mount Dennis, Keelsdale and Silverthorn                        4   
Don Mills                                                              26   
Dorset Park, Wexford Heights, Scarborough Town ...                      7   
Downsview                                                              17   
Dufferin, Dovercourt Village                                           16   
East Toronto, Broadview North (Old East York)                           3   
Eringate, Bloordale Gardens, Old Burnhamthorpe,...                      9   
Fairview, Henry Farm, Oriole                                           66   
First Canadian Place, Underground city                                100   
Forest Hill North & West, Forest Hill Road Park                         6   
Garden District, Ryerson                                              100   
Glencairn                                                               5   
Golden Mile, Clairlea, Oakridge                                         9   
Guildwood, Morningside, West Hill                                       7   
Harbourfront East, Union Station, Toronto Islands                     100   
High Park, The Junction South                                          25   
Hillcrest Village                                                       4   
Humber Summit                                                           2   
Humberlea, Emery                                                        1   
Humewood-Cedarvale                                                      4   
India Bazaar, The Beaches West                                         24   
Kennedy Park, Ionview, East Birchmount Park                             5   
Kensington Market, Chinatown, Grange Park                              62   
Kingsview Village, St. Phillips, Martin Grove G...                      4   
Lawrence Manor, Lawrence Heights                                       15   
Lawrence Park                                                           4   
Leaside                                                                33   
Little Portugal, Trinity                                               43   
Malvern, Rouge                                                          1   
Milliken, Agincourt North, Steeles East, L'Amor.

In [26]:
print('There are {} uniques categories.'.format(len(toronto_venues_df['Venue Category'].unique())))

There are 272 uniques categories.


In [27]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues_df[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

Yoga Studio  Accessories Store  Afghan Restaurant  Airport  \
0            0                  0                  0        0   
1            0                  0                  0        0   
2            0                  0                  0        0   
3            0                  0                  0        0   
4            0                  0                  0        0   

   Airport Food Court  Airport Gate  Airport Lounge  Airport Service  \
0                   0             0               0                0   
1                   0             0               0                0   
2                   0             0               0                0   
3                   0             0               0                0   
4                   0             0               0                0   

   Airport Terminal  American Restaurant  Antique Shop  Aquarium  Art Gallery  \
0                 0                    0             0         0            0   
1                 0                    0             0         0            0   
2                 0                    0             0         0            0   
3                 0                    0             0         0            0   
4                 0                    0             0         0            0   

   Art Museum  Arts & Crafts Store  Asian Restaurant  Athletics & Sports  \
0           0                    0                 0                   0   
1           0                    0                 0                   0   
2           0                    0                 0                   0   
3           0                    0                 0                   0   
4           0                    0                 0                   0   

   Auto Garage  Auto Workshop  BBQ Joint  Baby Store  Bagel Shop  Bakery  \
0            0              0          0           0           0       0   
1            0              0          0           0           0       0   
2            0              0          0           0           0       0   
3            0              0          0           0           0       0   
4            0              0          0           0           0       0   

   Bank  Bar  Baseball Field  Baseball Stadium  Basketball Court  \
0     0    0               0                 0                 0   
1     0    0               0                 0                 0   
2     0    0               0                 0                 0   
3     0    0               0                 0                 0   
4     0    0               0                 0                 0   

   Basketball Stadium  Beach  Bed & Breakfast  Beer Bar  Beer Store  \
0                   0      0                0         0           0   
1                   0      0                0         0           0   
2                   0      0                0         0           0   
3                   0      0                0         0           0   
4                   0      0                0         0           0   

   Belgian Restaurant  Bike Rental / Bike Share  Bike Shop  Bistro  \
0                   0                         0          0       0   
1                   0                         0          0       0   
2                   0                         0          0       0   
3                   0                         0          0       0   
4                   0                         0          0       0   

   Board Shop  Boat or Ferry  Bookstore  Boutique  Brazilian Restaurant  \
0           0              0          0         0                     0   
1           0              0          0         0                     0   
2           0              0          0         0                     0   
3           0              0          0         0                     0   
4           0              0          0         0                     0   

   Breakfast Spot  Brewery  Bridal Shop  Bubble Tea Shop  Building  \


In [28]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

Neighborhood  Yoga Studio  \
0                                           Agincourt     0.000000   
1                              Alderwood, Long Branch     0.000000   
2     Bathurst Manor, Wilson Heights, Downsview North     0.000000   
3                                     Bayview Village     0.000000   
4                   Bedford Park, Lawrence Manor East     0.000000   
5                                         Berczy Park     0.000000   
6                         Birch Cliff, Cliffside West     0.000000   
7        Brockton, Parkdale Village, Exhibition Place     0.041667   
8   Business reply mail Processing Centre, South C...     0.000000   
9   CN Tower, King and Spadina, Railway Lands, Har...     0.000000   
10                                Caledonia-Fairbanks     0.000000   
11              Canada Post Gateway Processing Centre     0.000000   
12                                          Cedarbrae     0.000000   
13                                 Central Bay Street     0.015625   
14                                           Christie     0.000000   
15                               Church and Wellesley     0.026667   
16            Clarks Corners, Tam O'Shanter, Sullivan     0.000000   
17    Cliffside, Cliffcrest, Scarborough Village West     0.000000   
18                     Commerce Court, Victoria Hotel     0.000000   
19                                         Davisville     0.000000   
20                                   Davisville North     0.000000   
21   Del Ray, Mount Dennis, Keelsdale and Silverthorn     0.000000   
22                                          Don Mills     0.000000   
23  Dorset Park, Wexford Heights, Scarborough Town...     0.000000   
24                                          Downsview     0.000000   
25                       Dufferin, Dovercourt Village     0.000000   
26      East Toronto, Broadview North (Old East York)     0.000000   
27  Eringate, Bloordale Gardens, Old Burnhamthorpe...     0.000000   
28                       Fairview, Henry Farm, Oriole     0.000000   
29             First Canadian Place, Underground city     0.000000   
30    Forest Hill North & West, Forest Hill Road Park     0.000000   
31                           Garden District, Ryerson     0.000000   
32                                          Glencairn     0.000000   
33                    Golden Mile, Clairlea, Oakridge     0.000000   
34                  Guildwood, Morningside, West Hill     0.000000   
35  Harbourfront East, Union Station, Toronto Islands     0.000000   
36                      High Park, The Junction South     0.000000   
37                                  Hillcrest Village     0.000000   
38                                      Humber Summit     0.000000   
39                                   Humberlea, Emery     0.000000   
40                                 Humewood-Cedarvale     0.000000   
41                     India Bazaar, The Beaches West     0.000000   
42        Kennedy Park, Ionview, East Birchmount Park     0.000000   
43          Kensington Market, Chinatown, Grange Park     0.000000   
44  Kingsview Village, St. Phillips, Martin Grove ...     0.000000   
45                   Lawrence Manor, Lawrence Heights     0.000000   
46                                      Lawrence Park     0.000000   
47                                            Leaside     0.000000   
48                           Little Portugal, Trinity     0.023256   
49                                     Malvern, Rouge     0.000000   
50  Milliken, Agincourt North, Steeles East, L'Amo...     0.000000   
51  Mimico NW, The Queensway West, South of Bloor,...     0.000000   
52                        Moore Park, Summerhill East     0.000000   
53       New Toronto, Mimico South, Humber Bay Shores     0.000000   
54           North Park, Maple Leaf Park, Upwood Park     0.000000   
55                 North Toronto West,  Lawrence Park     0.050000   
56                Northwest, West Humber - Clairvil

In [29]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0                     Lounge   0.2
1  Latin American Restaurant   0.2
2             Clothing Store   0.2
3             Breakfast Spot   0.2
4               Skating Rink   0.2


----Alderwood, Long Branch----
          venue  freq
0   Pizza Place  0.25
1  Skating Rink  0.12
2          Pool  0.12
3           Gym  0.12
4   Coffee Shop  0.12


----Bathurst Manor, Wilson Heights, Downsview North----
               venue  freq
0               Bank  0.10
1        Coffee Shop  0.10
2  Mobile Phone Shop  0.05
3     Sandwich Place  0.05
4     Ice Cream Shop  0.05


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Café  0.25
2                 Bank  0.25
3  Japanese Restaurant  0.25
4          Yoga Studio  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0  Italian Restaurant  0.09
1         Coffee Shop  0.09
2          Restaurant  0.09
3      Sandwich Place  0.09
4  

4    Department Store   0.2


----Kensington Market, Chinatown, Grange Park----
                           venue  freq
0                           Café  0.08
1  Vegetarian / Vegan Restaurant  0.06
2                    Coffee Shop  0.05
3                         Bakery  0.05
4          Vietnamese Restaurant  0.05


----Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens----
               venue  freq
0           Bus Line  0.25
1        Pizza Place  0.25
2     Sandwich Place  0.25
3  Mobile Phone Shop  0.25
4        Yoga Studio  0.00


----Lawrence Manor, Lawrence Heights----
                    venue  freq
0          Clothing Store  0.20
1       Accessories Store  0.13
2  Furniture / Home Store  0.13
3           Women's Store  0.07
4             Event Space  0.07


----Lawrence Park----
          venue  freq
0        Lawyer  0.25
1   Swim School  0.25
2      Bus Line  0.25
3          Park  0.25
4  Optical Shop  0.00


----Leaside----
                    venue  freq
0

               venue  freq
0  Indian Restaurant  0.09
1     Sandwich Place  0.09
2        Yoga Studio  0.05
3        Coffee Shop  0.05
4     Discount Store  0.05


----Toronto Dominion Centre, Design Exchange----
                 venue  freq
0          Coffee Shop  0.10
1                Hotel  0.08
2                 Café  0.07
3           Restaurant  0.04
4  Japanese Restaurant  0.03


----University of Toronto, Harbord----
                 venue  freq
0                 Café  0.15
1            Bookstore  0.06
2               Bakery  0.06
3                  Bar  0.06
4  Japanese Restaurant  0.06


----Victoria Village----
                   venue  freq
0           Intersection   0.2
1            Coffee Shop   0.2
2            Pizza Place   0.2
3  Portuguese Restaurant   0.2
4           Hockey Arena   0.2


----Westmount----
                       venue  freq
0               Intersection  0.14
1                Coffee Shop  0.14
2                Pizza Place  0.14
3  Middle Eastern Restaur

In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [31]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood      1st Most Common Venue  \
0                                        Agincourt  Latin American Restaurant   
1                           Alderwood, Long Branch                Pizza Place   
2  Bathurst Manor, Wilson Heights, Downsview North                Coffee Shop   
3                                  Bayview Village                       Café   
4                Bedford Park, Lawrence Manor East                Coffee Shop   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0        Clothing Store                Lounge        Breakfast Spot   
1          Skating Rink           Coffee Shop                   Gym   
2                  Bank      Sushi Restaurant     Mobile Phone Shop   
3                  Bank   Japanese Restaurant    Chinese Restaurant   
4        Sandwich Place    Italian Restaurant            Restaurant   

  5th Most Common Venue 6th Most Common Venue      7th Most Common Venue  \
0          Skating Rink  Colombian Restaurant    Comfort Food Restaurant   
1                   Pub                  Pool             Sandwich Place   
2              Pharmacy           Pizza Place  Middle Eastern Restaurant   
3         Women's Store   Distribution Center               Dessert Shop   
4         Grocery Store              Pharmacy                  Juice Bar   

         8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0  Eastern European Restaurant   Dumpling Restaurant              Drugstore  
1           Dim Sum Restaurant          Dance Studio          Deli / Bodega  
2                Deli / Bodega            Restaurant            Bridal Shop  
3           Dim Sum Restaurant                 Diner         Discount Store  
4                 Liquor Store     Indian Restaurant                    Pub

In [33]:
# set number of clusters
kclusters = int(len(toronto_df["Neighborhood"].unique()) / 4)
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=1).fit(toronto_grouped_clustering)

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_df.copy()
toronto_merged.rename(columns={'Neighbourhood':'Neighborhood'}, inplace=True)
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

Borough                                 Neighborhood  \
Postal Code                                                                  
M3A                North York                                    Parkwoods   
M4A                North York                             Victoria Village   
M5A          Downtown Toronto                    Regent Park, Harbourfront   
M6A                North York             Lawrence Manor, Lawrence Heights   
M7A          Downtown Toronto  Queen's Park, Ontario Provincial Government   

              Latitude  Longitude  Cluster Labels 1st Most Common Venue  \
Postal Code                                                               
M3A          43.753259 -79.329656            19.0                  Park   
M4A          43.725882 -79.315572             0.0           Coffee Shop   
M5A          43.654260 -79.360636            15.0           Coffee Shop   
M6A          43.718518 -79.464763            15.0        Clothing Store   
M7A          43.662301 -79.389494            15.0           Coffee Shop   

             2nd Most Common Venue       3rd Most Common Venue  \
Postal Code                                                      
M3A              Food & Drink Shop  Construction & Landscaping   
M4A          Portuguese Restaurant                Hockey Arena   
M5A                           Park                         Pub   
M6A              Accessories Store      Furniture / Home Store   
M7A                          Diner            Sushi Restaurant   

            4th Most Common Venue 5th Most Common Venue 6th Most Common Venue  \
Postal Code                                                                     
M3A                         Diner         Deli / Bodega      Department Store   
M4A                   Pizza Place          Intersection    Dim Sum Restaurant   
M5A                        Bakery                  Café            Restaurant   
M6A                 Women's Store   Arts & Crafts Store           Event Space   
M7A           Distribution Center                   Bar         Smoothie Shop   

            7th Most Common Venue 8th Most Common Venue 9th Most Common Venue  \
Postal Code                                                                     
M3A                  Dessert Shop    Dim Sum Restaurant        Discount Store   
M4A                  Dance Studio         Deli / Bodega      Department Store   
M5A                       Theater        Breakfast Spot           Yoga Studio   
M6A            Miscellaneous Shop              Boutique    Athletics & Sports   
M7A                      Beer Bar    Italian Restaurant      Sculpture Garden   

            10th Most Common Venue  
Postal Code                         
M3A                    Curling Ice  
M4A                   Dessert Shop  
M5A                     Shoe Store  
M6A                    Coffee Shop  
M7A                 Sandwich Place

In [39]:
# create map
map_clusters = folium.Map(location=[toronto_df["Latitude"][0], toronto_df["Longitude"][0]], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        #color=rainbow[cluster],
        fill=True,
        #fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters